In [4]:
import torch
import torchaudio
import torchmetrics
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import os
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Audio
from IPython.display import clear_output, display
import json
import librosa
import librosa.display as dsp
import math
import datetime
import time
import numpy as np
import ast

#Eigene Sachen
from data.data import *
from util.util import *
from util.datasets import *
from util.audio_processing import *
import util.plot as plot

In [3]:
model, util = torch.hub.load(
    repo_or_dir='snakers4/silero-vad',
    model='silero_vad',
    force_reload=True,
    trust_repo=False
)

#EVAL
model = model.eval()

The repository is already trusted.


Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to C:\Users\Jonas/.cache\torch\hub\master.zip


In [54]:
#MIN SAMPLE Länge für Silerio
SAMPLE_LENGTH = 512

#Accuracy
accuracy = torchmetrics.classification.BinaryAccuracy()

#Erstellt eigenes ChunkedDataset für Test
speakdataset_test_silerio = ChunkedDataset(speakdataset_test_unchunked, SAMPLE_LENGTH, SAMPLE_LENGTH, False)

#Ergebnis
pred_total = []
y_total    = []

#Iterriert über Sample
with torch.no_grad():
        
        #Iterriert Sample
        for x, y in iter(speakdataset_test_silerio):

                #Reset Model
                model.reset_states()

                #Prediction
                pred = torch.zeros_like(y)

                #Iterriert Samples
                for pos, chunk in enumerate(x):
                        
                        start_pos = pos * chunk.size(-1)
                        end_pos   = start_pos + chunk.size(-1)
                        pred[start_pos : end_pos] = model(chunk, SAMPLE_RATE).item()
                
                #Fügt Total an
                pred_total.append(pred)
                y_total.append(y)

#Tensor
y_total    = torch.hstack(y_total)
pred_total = torch.hstack(pred_total)

#Print für Accuracy
print("Accuracy auf Batch:", accuracy(pred_total, y_total))

Accuracy auf Batch: tensor(0.8942)
